In [ ]:
def neighbor_joining(df):
    # Convert the DataFrame to a numpy matrix
    dist_matrix = df.to_numpy()
    taxa = list(df.index)
    tree = {}

    while len(taxa) > 2:
        n = len(taxa)
        Q_matrix = np.zeros((n, n))
        print(n)
        
        # Compute the Q matrix
        sumi = [np.sum(dist_matrix[i]) for i in range(n)]
        
        for i in range(n):
            for j in range(n):
                if i != j:
                    Q_matrix[i][j] = (n - 2) * dist_matrix[i][j] - sumi[i] - sumi[j]
        
        # Find the pair with the smallest Q value
        min_i, min_j = np.unravel_index(np.argmin(Q_matrix), Q_matrix.shape)
        
        # Compute the distance to the new node
        new_dist = (dist_matrix[min_i][min_j] +  (sumi[min_i] - sumi[min_j]) / (n - 2)) / 2
        
        # Update the tree
        new_node = taxa[min_i] + '.' + taxa[min_j]
        tree[new_node] = {taxa[min_i]: new_dist,
                          taxa[min_j]: dist_matrix[min_i][min_j] - new_dist}
        
        # Update the distance matrix
        new_row = [(dist_matrix[min_i][k] + dist_matrix[min_j][k] - dist_matrix[min_i][min_j]) / 2 for k in range(n) if k != min_i and k != min_j]
        new_dist_matrix = np.delete(dist_matrix, [min_i, min_j], axis=0)
        new_dist_matrix = np.delete(new_dist_matrix, [min_i, min_j], axis=1)
        new_dist_matrix = np.vstack((new_dist_matrix, new_row))
        new_row.append(0)
        new_dist_matrix = np.column_stack((new_dist_matrix, new_row))
        
        # Update the taxa list
        new_taxa = taxa[:]
        new_taxa.remove(taxa[min_i])
        new_taxa.remove(taxa[min_j])
        new_taxa.append(new_node)
        
        # Update the distance matrix and taxa list
        dist_matrix = new_dist_matrix
        taxa = new_taxa
        
    # Connect the final two nodes
    tree[taxa[1]][taxa[0]] = dist_matrix[0, 1]
    
    return tree